In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

## Use a Keras layer

In [ ]:
layer = keras.layers.Dense(
    2,
    activation="relu",
    bias_initializer="zeros",
)
layer(x)

## Write your custom layer

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.w = self.add_weight(
            shape=[input_dim, self.units], initializer="glorot_uniform",
        )
        self.b = self.add_weight(
            shape=[self.units,], initializer="zeros"
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


# Different ways of building the model

- Sequential API
- [**Preferred**] Functional API
- Subclassing model

In [ ]:
# Sequential model
sequential_model = keras.Sequential([
    keras.Input([28, 28, 3]),
    keras.layers.Conv2D(8, 2),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(2),
    keras.layers.Softmax(),
])

print(sequential_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 27, 27, 8)         104       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 13, 13, 8)        0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 1352)              0         
                                                                 
 dense_3 (Dense)             (None, 2)                 2706      
                                                                 
 softmax_3 (Softmax)         (None, 2)                 0         
                                                                 
Total params: 2,810
Trainable params: 2,810
Non-trainable params: 0
____________________________________________________

In [ ]:
# Functional model
inputs = keras.Input([28, 28, 3])
x = keras.layers.Conv2D(8, 2)(inputs)
x = keras.layers.MaxPool2D(2)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(2)(x)
outputs = keras.layers.Softmax()(x)

functional_model = keras.Model(inputs=inputs, outputs=outputs)
print(functional_model.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 28, 28, 3)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 27, 27, 8)         104       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 8)        0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 1352)              0         
                                                                 
 dense_4 (Dense)             (None, 2)                 2706      
                                                                 
 softmax_4 (Softmax)         (None, 2)                 0         
                                                           

In [ ]:
# Subclassing `keras.Model`

class MyModel(keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._conv = keras.layers.Conv2D(8, 2)
        self._max_pool = keras.layers.MaxPool2D(2)
        self._flatten = keras.layers.Flatten()
        self._dense = keras.layers.Dense(2)
        self._softmax = keras.layers.Softmax()

    def call(self, inputs):
        x = self._conv(inputs)
        x = self._max_pool(x)
        x = self._flatten(x)
        x = self._dense(x)
        return self._softmax(x)

subclass_model = MyModel()
subclass_model.build([None, 28, 28, 3])
subclass_model.summary()

# Models are callable.

`keras.Model` instances are also callable. You can call models as if it is a function, no matter which way you used to build the model.

In [ ]:
sample_data = tf.random.uniform([1, 28, 28, 3])

print(sequential_model(sample_data))
print(functional_model(sample_data))
print(subclass_model(sample_data))

tf.Tensor([[0.65723634 0.34276363]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.2294048  0.77059513]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.5731103 0.4268897]], shape=(1, 2), dtype=float32)


In [ ]:
variable_names = [v.name for v in functional_model.trainable_variables]
print("\n".join(variable_names))

conv2d/kernel:0
conv2d/bias:0
dense/kernel:0
dense/bias:0


# Train the model

Now we have defined the model. For actual training, we need the following pieces:
- Data
- Loss/Target function
- Optimizer

In [ ]:
# Prepare data for a 2-class classification.
data = tf.random.uniform([20, 28, 28, 3])
label = tf.convert_to_tensor(np.random.randint(2, size=20))

In [ ]:
# Let's use functional model defined above.
model = functional_model

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
model.fit(
    data,
    label,
    batch_size=5,
    epochs=2,
)

Epoch 1/2
4/4 [==============================] - 1s 5ms/step - loss: 0.7464 - sparse_categorical_accuracy: 0.4000
Epoch 2/2
4/4 [==============================] - 0s 5ms/step - loss: 0.7288 - sparse_categorical_accuracy: 0.5500


# Eager mode and graph mode

Debug in eager mode, put actual job in graph mode.

In [ ]:
# Eager mode
print(model(data[0:1, ...]))

# Graph mode
print(model.predict(data[0:1, ...]))

tf.Tensor([[0.293789 0.706211]], shape=(1, 2), dtype=float32)
1/1 [==============================] - 0s 25ms/step
[[0.293789 0.706211]]


In [ ]:
@tf.function(jit_compile=True)
def forward_pass(x):
    outputs = model(x)
    print(outputs)
    return outputs

print(forward_pass(data[0:1, ...]))

Tensor("my_model/softmax_5/Softmax:0", shape=(1, 2), dtype=float32)
tf.Tensor([[0.2937891 0.7062109]], shape=(1, 2), dtype=float32)


In [ ]:
# You can also run `fit`, `predict` in eager mode.
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    run_eagerly=True
)

# Run `predict` in eager mode by setting `run_eagerly` above.
print(model.predict(data[0:1, ...]))

# Write your custom training loop

We encourage using `fit` for training/finetuning as much as possible, but it's still important to write you custom training loop. For example, if you want to debug the training loop, or have a better control over each step, you can use custom training loop.

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((data, label))
train_data = train_data.batch(5).cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam(0.001)
metric = keras.metrics.SparseCategoricalAccuracy()

In [ ]:
@tf.function
def train_step(data):
    x, y = data
    with tf.GradientTape() as tape:
        outputs = model(x)
        loss = loss_fn(y, outputs)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    metric.update_state(y, outputs)

    return loss


for i, data in enumerate(train_data):
    loss = train_step(data)
    print(f"Batch {i}, loss: {loss:.3f}, accuracy: {metric.result():.3f}")

Batch 0, loss: 0.522, accuracy: 0.800
Batch 1, loss: 0.509, accuracy: 0.900
Batch 2, loss: 0.761, accuracy: 0.733
Batch 3, loss: 0.976, accuracy: 0.600


# Save your model.

In [ ]:
# Save the weights.
model.save_weights("weights")

In [ ]:
subclass_model.load_weights("weights")

In [ ]:
tf.reduce_sum(subclass_model._conv.kernel - model.get_layer("conv2d_5").kernel)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

# Use callbacks to control your training

`keras.callbacks` provides the ability to control your training loop. For example, you can use callbacks to save your checkpoints at a certain frequency.

In [ ]:
!mkdir checkpoints

In [ ]:
checkpoint_filepath = 'checkpoints/checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor="loss",
)

model.fit(
    data,
    label,
    batch_size=5,
    epochs=3,
    callbacks=[model_checkpoint_callback]
)

Epoch 1/3
1/4 [======>.......................] - ETA: 0s - loss: 0.2408 - sparse_categorical_accuracy: 1.0000
Epoch 1: saving model to checkpoints/checkpoint
4/4 [==============================] - 0s 12ms/step - loss: 0.2213 - sparse_categorical_accuracy: 1.0000
Epoch 2/3
1/4 [======>.......................] - ETA: 0s - loss: 0.2299 - sparse_categorical_accuracy: 1.0000
Epoch 2: saving model to checkpoints/checkpoint
4/4 [==============================] - 0s 11ms/step - loss: 0.2116 - sparse_categorical_accuracy: 1.0000
Epoch 3/3
1/4 [======>.......................] - ETA: 0s - loss: 0.2026 - sparse_categorical_accuracy: 1.0000
Epoch 3: saving model to checkpoints/checkpoint
4/4 [==============================] - 0s 14ms/step - loss: 0.1898 - sparse_categorical_accuracy: 1.0000
